In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import mdtraj as md
import mosaic
from tqdm.auto import tqdm, trange
#import pickle 

plt.rcParams.update({'font.size': 18})

Loading the trajectory

In [ ]:
### THE TRAJECTORY IS CALLED C1, BUT THIS IS THE NEW C2! 
### THE ONE WHERE THE LIGAND LEAVES FROM TOP OF PROT

In [ ]:
traj= md.load("../C1_TRAJ_it986_seg350.nc", top="../ired.prmtop")
traj1=traj[0:1400] ## bound + transient state
traj,traj1

Creation of a new trajectory containing only the heavy atoms of the protein 

In [ ]:
aln = traj1.top.select('protein and not (element H or resid 0)')
traj_prot = traj1.atom_slice(aln)
traj_prot

## Calculation of all contacts between the heavy atoms of the protein.

*dist_idx* is an array containing index pairs (indexed to 0) of residues from which to compute contacts. 
The string 'all' will select all pairs of residues separated by two or more residues (i.e., pairs i-i+1 and i-i+2 will be excluded).

*dist_pairs* is an array containing the value of distances between pairs of residues in nm


In [ ]:
%%time 
dist_pairs, dist_idx = md.compute_contacts(traj_prot, contacts='all', scheme='closest-heavy')

## Computation of the Chi1 angles 
The angle is expressed in *radiants*. In this specific case:
- _chi1_idx(n_chi, 4)_ contains the indices of the atoms involved in each of the chi1 dihedral angles.
- _chi1(n_frames, n_chi)_ contains the value of the dihedral angle for each of the angles in each of the frames.

In [ ]:
chi1_idx,chi1=md.compute_chi1(traj_prot)

In [ ]:
print(chi1_idx.shape)
print(chi1.shape) 

## Selection of relevant contacts

We will now filter these two matrices. We need to consider only those pairs of residues that are in contact (distance <= 0.45 nm) for at least 10% of the total simulation time.

In the first step we will create a shape matrix(n_frames,n_contacs) filled with values 0 or 1. 
If the distance is less than 0.45 nm, the value will be 0, otherwise 1.

In [ ]:
arr=np.zeros((dist_pairs.shape[0],dist_pairs.shape[1] ))
arr_cont=np.zeros((dist_pairs.shape[0],dist_pairs.shape[1] ))

for i in trange(dist_pairs.shape[0]):                      
    for j in np.arange(dist_pairs.shape[1]):
        if dist_pairs[i,j] <= 0.45:
            arr_cont[i,j] = 1

To have an idea of the persistance of contacts during the simulation, we calculate the mean which will assume a value between 0 and 1

In [ ]:
mean=np.average(arr_cont, axis=0)

Now we create a list with the indeces of the pairwise contacts that are conserved for at least the 10% of the simulation time.
These residues will be used for the Leiden Clustering

In [ ]:
list_id=[]
for idx, cont in enumerate(mean):                       
    if cont > 0.1:
        list_id.append(idx)

In [ ]:
len(list_id)

At this point we need to generate the input for Leiden Clustering. 
We need to filter the contact array *dist_pairs*, keeping only those columns (distances) whose index is contained in *list_id*.

This step is the longest, so it might be a good idea to save the generated matrix

In [ ]:
dist_idx.shape, dist_pairs.shape[0]

In [ ]:
final=np.zeros((dist_pairs.shape[0], len(list_id)))   
for i in trange(dist_pairs.shape[0]):
    count = 0
    for j in np.arange(dist_pairs.shape[1]):
        if j in list_id:
            final[i,count]=dist_pairs[i,j]
            count= count+1
            
#np.save('path2_selected_contacts.npy', final)

To retrieve the indices of the residue pairs, we must also filter the *dist_idx* file keeping only the pairs that are included in the *list_id*.

In [ ]:
final_idx=[]
for i in trange(dist_idx.shape[0]):
    if i in list_id:
        final_idx.append(dist_idx[i])
final_id=np.array(final_idx)
#np.save('path2_selected_contacts_idx.npy', final_id)

In [ ]:
#final=np.load('../../final_leiden/path2_selected_contacts.npy')
#final_idx=np.load('../../final_leiden/path2_selected_contacts_idx.npy')

## Contacts + chi1 angles

Now I am creating an array by concatenating the arrays of distances and chi angles. The first has a shape of (3001, 346) and the second has a shape of (3001, 105). So the resulting array has a form of (3001, 451). 

Note that we cannot create the *total_idx* because the elements of the final_idx are pairs of residues, while the elements of the chi1_idx are the indices of the 4 atoms defining the chi1 angle. Therefore, the shapes of the two arrays are different and cannot be concatenated.

In [ ]:
total=np.concatenate((chi1,final), axis=1)

In [ ]:
print(total.shape)
print(final.shape)
print(chi1.shape)

In [ ]:
print(chi1_idx.shape)
print(final_id.shape)

## Leiden Clustering

In the final step, the *final* file is passed as an argument to the *leiden* function to obtain the clusters

In [ ]:
def leiden(trax, filename='leiden_'):
    '''
    docstring for leiden clustering 
    '''
    
    sim = mosaic.Similarity(metric='correlation')
    sim.fit(trax)
    clust=mosaic.Clustering(mode='CPM', resolution_parameter=0.5)
    clust.fit(sim.matrix_)
    clusters=clust.clusters_
    #clustered_X=clust.matrix_
    #ticks=clust.ticks_
    #labels=clust.labels_
    #with open(f'/home/riccardo529cp/compchem/HIF/analysis/leiden_cont_protein/{filename}.pickle', "wb") as file:        
    #    pickle.dump (clustered_X, file)
    return clusters #, clustered_X, ticks, labels

In [ ]:
%%time
path2_clusters = leiden(total, filename='path1')  ## cleaned

As reported in previous work, we will consider clusters containing more than 3 coordinates as "main clusters"

In [ ]:
main_clusters=[]
for i in path2_clusters:
    if len(i) >= 3:
        main_clusters.append(i)
main_clust=np.asarray(main_clusters)

In [ ]:
print("The correlated residues were clustered in", main_clust.shape[0], "groups.")
#print(main_clust)

Now the goal is to retrieve the indices of the residues in each cluster so that we can display them in VMD. However, as explained above, we cannot create the *total_idx* array. 

So we convert the clusters of four atoms forming each chi1 corner into the index of the residue (to make it consistent with the final_idx).

In [ ]:
chi_atoms=chi1_idx[:,0]
chi_residues=[]

for i in chi_atoms:
    chi_residues.append((traj_prot.top.atom(i).residue.index)-1)
    
chi_res=np.array(chi_residues)

In [ ]:
len(chi_residues)

Here we want to put all the elements of a specific cluster into a list (contacts or corners). Since the first 105 elements of the *total* array (from which *main_clust* is derived) are corners and the others are contacts, we will look for the indices of the residuals in the *chi_res* array and then in the *final_idx* array.

In [ ]:
# Initialize a dictionary to hold the clusters
clusters = {f'cluster{i+1}': [] for i in range(main_clust.shape[0])}

# Loop through each cluster in main_clust
for idx, cluster in enumerate(main_clust):
    for i in cluster:
        # Determine which array to append to based on the value of i
        if i < len(chi_residues):
            clusters[f'cluster{idx+1}'].append(chi_res[i])
        else:
            j = i - len(chi_residues)
            clusters[f'cluster{idx+1}'].append(final_idx[j])

# Now clusters is a dictionary with each cluster data

In [ ]:
clusters['cluster1']

Residues in clusters are 0-indexed. To visualize them, we want them to be 2-indexed (since ligand is residue #1) 

In [ ]:
vmd_resid = {f'vmd_resid_cluster{i+1}': [] for i in range(main_clust.shape[0])}

for i in range(main_clust.shape[0]):
    vmd_resid[f'vmd_resid_cluster{i+1}'] = np.array([j+2 for j in clusters[f'cluster{i+1}']])

In [ ]:
vmd_resid

In [ ]:
for i in range(main_clust.shape[0]):
    if(len(vmd_resid[f'vmd_resid_cluster{i+1}']) > 3):
        print(i,vmd_resid[f'vmd_resid_cluster{i+1}'])
        print('\n')

# Script for automatic generation of tcl script for displaying networks of correlated motions

- In the first part of the cell, I extracted the CA coordinates from the reference pdb structure (ref_real.pdb).
- In the second part, I added all the necessary lines to display the cylinders connecting the correlated residuals to the motions. For some reason, the i-th element of the *vmd_resid_cluster1* corresponds to the (i - 2)th element of the *xyz* dataset.

In [ ]:
pdb_file = "reference.pdb"

with open(pdb_file, "r") as pdb:
    pdb_lines = pdb.readlines()
    xyz = [k.rsplit()[5:8] for k in pdb_lines[1:] if "CA" in k]

with open('allo_C2.tcl', "w") as tcl:
    tcl.write("draw delete all \n")
    clusters = [
        
        (vmd_resid['vmd_resid_cluster1'], "red"),
        (vmd_resid['vmd_resid_cluster2'], "cyan"),
        (vmd_resid['vmd_resid_cluster3'], "yellow"),
        (vmd_resid['vmd_resid_cluster4'], "blue2")
    ]
    for i, (cluster_idx, color) in enumerate(clusters, start=1):
        tcl.write(f"\n #cluster{i} \n")
        tcl.write("draw material Diffuse \n")
        tcl.write(f"draw color {color} \n")
        for j, idx in enumerate(cluster_idx):
            if isinstance(idx, np.ndarray):
                x1, y1, z1 = xyz[idx[0] - 2]
                x2, y2, z2 = xyz[idx[1] - 2]
                tcl.write(f"draw cylinder {{{x1} {y1} {z1}}} {{{x2} {y2} {z2}}} radius 0.3 \n")